<p style="font-size:30px;">
Поиск и предложение похожих товаров
</p>

В нашем распоряжении данные о товарах от одного из крупнейших маркетплейсов страны.
Разработаем алгоритм, который для списка товаров `validation.csv` предложит 5 вариантов наиболее похожих товаров из другого списка `base.csv`. Оценка качества алгоритма будем проводить по метрике `accuracy@5`.

Данные:

- *`base.csv`* - анонимизированный набор товаров. Каждый товар представлен как уникальный id (0-base, 1-base, 2-base) и вектор признаков размерностью 72.

- *`target.csv`* - обучающий датасет. Каждая строчка - один товар, для которого известен уникальный id (0-query, 1-query, …) , вектор признаков и `id` товара из *`base.csv`*, который максимально похож на него (по мнению экспертов).

- *`validation.csv`* - датасет с товарами (уникальный id и вектор признаков), для которых надо найти наиболее близкие товары из *`base.csv`*

- *`validation_answer.csv`* - правильные ответы к файлу `validation.csv`.

<p style="font-size:25px;">
Часть II. Повышение качества поиска с использованием ранжирующей модели
</p>

Метод поиска схожих векторов методом кластеризации из библиотеки Faiss выдает на тренировочной и валидационной выборках результат 70%. 

Исследуем возможность повышения точности поиска путем последующего применения ранжирующей модели машинного обучения. Для этого также вычислены топ 50, 100, 200, 400 ближайших векторов.

С увеличением числа ближайших векторов до 50 точность выросла ориентировочно на 8%. При увеличении числа ближайших векторов в интервале от 50 до 400 точность повышается незначительно, на 1-1,5% для каждого из интервалов (50:100, 100:200, 200:400).  

Обучим и применим ранжирующую модель на выбоках из топ 50, 100, 200 и 400 векторов.

In [1]:
!pip install optuna

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import faiss

from sklearn.preprocessing import StandardScaler, MinMaxScaler, \
                                  RobustScaler, PowerTransformer, \
                                  QuantileTransformer
from sklearn.metrics import make_scorer 

from catboost import CatBoostClassifier, Pool, cv 
import optuna

import matplotlib.pyplot as plt
import seaborn as sns
from joblib import dump, load

import datetime
# from google.colab import drive
# drive.mount('/content/drive/')

%matplotlib inline

## Загрузка данных

In [3]:
dict_base = {}
for i in range(72):
    dict_base[str(i)] = 'float32'
dict_train = dict_base.copy()
dict_train['Target'] = 'str'

In [4]:
%%time
all_base = pd.read_csv('./datasets/base.csv', index_col=0, dtype=dict_base)
#all_base = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/projects/matching/datasets/base.csv', index_col=0)

CPU times: user 20.1 s, sys: 799 ms, total: 20.9 s
Wall time: 21 s


In [5]:
all_train = pd.read_csv('./datasets/train.csv', index_col=0, dtype=dict_train)
valid_features = pd.read_csv('./datasets/validation.csv', index_col=0, dtype=dict_base)
valid_target = pd.read_csv('./datasets/validation_answer.csv', index_col=0)
# all_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/projects/matching/datasets/train.csv', index_col=0)
# valid_features = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/projects/matching/datasets/validation.csv', index_col=0)

In [6]:
base_index = {k: v for k, v in enumerate(all_base.index.to_list())}

train_target = all_train['Target']
train_features = all_train.drop('Target', axis=1)

def accuracy_top_n(target_features, indices):
    acc = 0
    for target, el in zip(target_features.values.tolist(), indices.tolist()):
        acc += int(target in [base_index[r] for r in el])

    return 100 * acc / len(indices)

In [7]:
col_list = ['6', '21', '25', '33', '44', '59', '65', '70']

Загрузим результат поиска ближайших векторов методом кластеризации для векторов из тестовой и валидационной выборок

In [8]:
train_indices = np.load('./dumps/train_indices_100.npy')
valid_indices = np.load('./dumps/valid_indices_100.npy')

In [9]:
accuracy_top_n(train_target, train_indices)

79.976

In [10]:
accuracy_top_n(valid_target['Expected'], valid_indices)

79.857

## Сформируем данные для обучения модели

In [11]:
train_indices.shape

(100000, 100)

Учитывая значительный объем данных определим максимальный размер блока данных в 1 млн. векторов

In [12]:
i = int(1000000 / train_indices.shape[1])
j = train_indices.shape[1]
print('i=', i, 'j=', j)

i= 10000 j= 100


**Формирование тренировочной выборки**

In [13]:
rank_train_sub_indices = train_indices[:i, :j].ravel()
len(rank_train_sub_indices)

1000000

In [14]:
rank_train_sub_base = all_base.iloc[rank_train_sub_indices, :].reset_index()
rank_train_sub_base.shape

(1000000, 73)

In [15]:
rank_train = all_train.iloc[np.repeat(range(i), j), :].reset_index(drop=True)
rank_train.shape

(1000000, 73)

In [16]:
rank_train = pd.concat([rank_train, rank_train_sub_base], axis=1, ignore_index=True)
rank_train.shape

(1000000, 146)

In [17]:
rank_train['target'] = np.where(rank_train[72] == rank_train[73], 1,  0)

Доля объектов положительного класса в тренировочной выборке

In [18]:
rank_train['target'].mean()

0.008042

In [19]:
rank_train = rank_train.drop([72, 73], axis=1)

### Обучение ранжирующей модели

#### Подбор гиперпараметров

In [20]:
train_pool = Pool(data=rank_train.drop(['target'], axis=1), 
                  label=rank_train['target'])

In [21]:
def cb_objective(trial: optuna.Trial, train_pool):

    params_set = {
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.3, log=True),
        'eval_metric': 'Recall',
        'random_seed': 42,
        'verbose': 1000
    }

    cb = CatBoostClassifier(**params_set)

    cb.fit(train_pool,
           early_stopping_rounds=200,
           verbose=1000)

    score = cb.best_score_['learn']['Recall'] #['RecallAt:top=5'] #['MAP']

    return score

In [22]:
cb_study = optuna.create_study(study_name='cb_study',
                               pruner=optuna.pruners.MedianPruner(n_warmup_steps=1),
                               direction='maximize')

[I 2023-09-29 23:07:48,146] A new study created in memory with name: cb_study


In [23]:
%%time
cb_study.optimize(lambda trial: cb_objective(trial, train_pool),
                  n_trials=10,
                  show_progress_bar=True);

  0%|          | 0/10 [00:00<?, ?it/s]

0:	learn: 0.0000000	total: 260ms	remaining: 4m 19s
999:	learn: 0.5726188	total: 3m 10s	remaining: 0us
[I 2023-09-29 23:11:00,699] Trial 0 finished with value: 0.5729917930862969 and parameters: {'learning_rate': 0.12463489586445073}. Best is trial 0 with value: 0.5729917930862969.
0:	learn: 0.0000000	total: 218ms	remaining: 3m 37s
999:	learn: 0.5823178	total: 3m 12s	remaining: 0us
[I 2023-09-29 23:14:15,379] Trial 1 finished with value: 0.5823178313852275 and parameters: {'learning_rate': 0.12391436293465248}. Best is trial 1 with value: 0.5823178313852275.
0:	learn: 0.0000000	total: 220ms	remaining: 3m 39s
999:	learn: 0.5533449	total: 3m 15s	remaining: 0us
[I 2023-09-29 23:17:33,168] Trial 2 finished with value: 0.5534692862472022 and parameters: {'learning_rate': 0.10880883068359615}. Best is trial 1 with value: 0.5823178313852275.
0:	learn: 0.0000000	total: 211ms	remaining: 3m 31s
999:	learn: 0.5644118	total: 3m 14s	remaining: 0us
[I 2023-09-29 23:20:49,043] Trial 3 finished with va

top_n=400 [I 2023-09-24 03:51:18,579] ['RecallAt:top=5'] Trial 6 finished with value: 1.0 and parameters: {'learning_rate': 0.20635329594703775}. Best is trial 6 with value: 1.0.

top_n=400 [I 2023-09-24 12:13:42,586] ['MAP'] Trial 7 finished with value: 0.8376 and parameters: {'learning_rate': 0.23673897995559087}. Best is trial 7 with value: 0.8376.

top_n=200 [I 2023-09-24 19:26:41,401] ['MAP'] Trial 4 finished with value: 0.8139082204125878 and parameters: {'learning_rate': 0.18044027964226522}. Best is trial 4 with value: 0.8139082204125878.

top_n=100 [I 2023-09-25 00:44:42,799] Trial 0 finished with value: 0.7971334687318913 and parameters: {'learning_rate': 0.20577118484210952}. Best is trial 0 with value: 0.7971334687318913.

In [27]:
cb_study.best_params

{'learning_rate': 0.25270456338028674}

In [42]:
%%time
model = CatBoostClassifier(**cb_study.best_params, 
                           eval_metric='Recall',
                           random_seed=42)
model.fit(train_pool, verbose=100)

Learning rate set to 0.196759
0:	learn: 0.0000000	total: 214ms	remaining: 3m 33s
100:	learn: 0.2203475	total: 22.9s	remaining: 3m 24s
200:	learn: 0.3979503	total: 42.3s	remaining: 2m 48s
300:	learn: 0.4773153	total: 1m 1s	remaining: 2m 22s
400:	learn: 0.5351831	total: 1m 19s	remaining: 1m 59s
500:	learn: 0.5605549	total: 1m 38s	remaining: 1m 37s
600:	learn: 0.5810524	total: 1m 56s	remaining: 1m 17s
700:	learn: 0.5996750	total: 2m 14s	remaining: 57.4s
800:	learn: 0.6166729	total: 2m 32s	remaining: 37.9s
900:	learn: 0.6365454	total: 2m 50s	remaining: 18.7s
999:	learn: 0.6527934	total: 3m 7s	remaining: 0us
CPU times: user 12min 18s, sys: 4.82 s, total: 12min 23s
Wall time: 3min 9s


In [43]:
model.best_score_

{'learn': {'Recall': 0.6532933383327084, 'Logloss': 0.009881399772343528}}

In [86]:
# model.save_model('./dumps/trained_cb_class_model_ranker_{}_{}_{}'
#                 .format(j, 
#                         model.get_params()['eval_metric'], 
#                         model.best_score_['learn']['Recall']))

In [ ]:
# model = CatBoostClassifier()
# model.load_model('./dumps/trained_cb_class_model_ranker_100_Recall_0.6480974881870182')

#### Дообучение модели

In [45]:
for bins, t in enumerate(range(i, train_indices.shape[0], i)):
    print(t, t + i)
    
    start_time = datetime.datetime.now()
    
    rank_train_sub_indices = train_indices[t:i + t, :j].ravel()

    rank_train_sub_base = all_base.iloc[rank_train_sub_indices, :].reset_index()

    #all_valid = pd.concat([valid_features, valid_target], axis=1)
    rank_train = all_train.iloc[np.repeat(range(t, i + t), j), :].reset_index(drop=True)
    
    rank_train = pd.concat([rank_train, rank_train_sub_base], axis=1, ignore_index=True)

    rank_train['target'] = np.where(
        rank_train[72] == rank_train[73], 
        1,  
        0)
    print(rank_train['target'].mean()) 
    
    rank_train = rank_train.drop([72, 73], axis=1)
    
    train_pool = Pool(data=rank_train.drop(['target'], axis=1), 
                      label=rank_train['target'])
    
    model.fit(train_pool, init_model=model, verbose=1000)
    
    end_time = datetime.datetime.now()
    print(round((end_time - start_time).total_seconds() / 60, 2), 'мин') 


10000 20000
0.00799
Learning rate set to 0.196759
0:	learn: 0.5232791	total: 180ms	remaining: 2m 59s
999:	learn: 0.6689612	total: 2m 59s	remaining: 0us
3.05 мин
20000 30000
0.008017
Learning rate set to 0.196759
0:	learn: 0.5714108	total: 180ms	remaining: 3m
999:	learn: 0.6750655	total: 2m 57s	remaining: 0us
3.03 мин
30000 40000
0.007974
Learning rate set to 0.196759
0:	learn: 0.5871583	total: 153ms	remaining: 2m 33s
999:	learn: 0.6882368	total: 2m 56s	remaining: 0us
3.01 мин
40000 50000
0.008022
Learning rate set to 0.196759
0:	learn: 0.5964847	total: 191ms	remaining: 3m 11s
999:	learn: 0.6930940	total: 2m 57s	remaining: 0us
3.04 мин
50000 60000
0.008055
Learning rate set to 0.196759
0:	learn: 0.6095593	total: 203ms	remaining: 3m 23s
999:	learn: 0.6967101	total: 2m 56s	remaining: 0us
3.03 мин
60000 70000
0.007975
Learning rate set to 0.196759
0:	learn: 0.6092790	total: 181ms	remaining: 3m
999:	learn: 0.7000627	total: 2m 58s	remaining: 0us
3.06 мин
70000 80000
0.007963
Learning rate se

In [46]:
model.best_score_

{'learn': {'Recall': 0.7930258717660292, 'Logloss': 0.006331737518687488}}

In [90]:
# model.save_model('./dumps/all_trained_cb_class_model_ranker_{}_{}_{}'
#                 .format(j, 
#                         model.get_params()['eval_metric'], 
#                         model.best_score_['learn']['Recall']))

In [37]:
#model = CatBoostClassifier()
#model.load_model('./dumps/all_trained_cb_class_model_ranker_100_Recall_0.7007874015748031')

### Предсказание ранжирующей модели на валидационной выборке

In [59]:
print(i, j)

10000 100


Проведем предсказание рандирующей модели для каждого блока по 10000 векторов из исходной валидационной выборки

In [47]:
all_predicted = np.array([])
for bins, t in enumerate(range(0, valid_indices.shape[0], i)):
    print('part: ', t, t + i)
    rank_valid_sub_indices = valid_indices[t:i+t, :j].ravel()

    rank_valid_sub_base = all_base.iloc[rank_valid_sub_indices, :].reset_index()
    rank_valid_sub_base.shape

    all_valid = pd.concat([valid_features, valid_target], axis=1)
    rank_valid = all_valid.iloc[np.repeat(range(t, i + t), j), :].reset_index(drop=True)
    rank_valid.shape

    rank_valid = pd.concat([rank_valid, rank_valid_sub_base], axis=1, ignore_index=True)

    rank_valid = rank_valid.drop([72, 73], axis=1)

    valid_pool = Pool(data=rank_valid)

    predicted = model.predict(valid_pool, prediction_type='Probability')
    
    all_predicted = np.append(all_predicted, predicted[:, 1])
    
    predicted_array = predicted[:, 1].reshape(i, j)
    
    best_predicted_indices = (-predicted_array).argsort()[:, :5]
    
    best_predicted_neighbors = []
    
    for k, index in enumerate(best_predicted_indices):
        best_predicted_neighbors = np.concatenate(
            (best_predicted_neighbors, 
             valid_indices[t:i + t, :j][k, index]), 
            axis=0)
        
    best_predicted_neighbors = best_predicted_neighbors.reshape(i, 5)
    
    print('accuracy@5 после ранжирования модели: ', 
          accuracy_top_n(valid_target.iloc[t:i + t].reset_index(drop=True)['Expected'], 
                         best_predicted_neighbors))
    

part:  0 10000
accuracy@5 после ранжирования модели:  74.18
part:  10000 20000
accuracy@5 после ранжирования модели:  73.21
part:  20000 30000
accuracy@5 после ранжирования модели:  74.25
part:  30000 40000
accuracy@5 после ранжирования модели:  73.7
part:  40000 50000
accuracy@5 после ранжирования модели:  74.01
part:  50000 60000
accuracy@5 после ранжирования модели:  73.78
part:  60000 70000
accuracy@5 после ранжирования модели:  73.88
part:  70000 80000
accuracy@5 после ранжирования модели:  74.11
part:  80000 90000
accuracy@5 после ранжирования модели:  73.72
part:  90000 100000
accuracy@5 после ранжирования модели:  73.87


**Найдем топ 5 наиболее подходящих векторов с использованием ранжирующей модели**

In [48]:
predicted_array = all_predicted.reshape(valid_indices.shape[0], j)
predicted_array.shape

(100000, 100)

In [49]:
best_predicted_indices = (-predicted_array).argsort()[:, :5]
best_predicted_indices

array([[77, 21, 20, 19, 40],
       [22, 30, 34, 40, 51],
       [ 0,  3,  1,  6, 54],
       ...,
       [26,  3,  4, 39,  5],
       [ 0,  4,  3, 82, 49],
       [99, 43, 41, 66, 86]])

In [50]:
best_predicted_neighbors = []
for k, index in enumerate(best_predicted_indices):
    best_predicted_neighbors = np.concatenate((best_predicted_neighbors, valid_indices[k, index]), axis=0)

In [51]:
best_predicted_neighbors = best_predicted_neighbors.reshape(valid_indices.shape[0], 5)

In [52]:
print('accuracy@5 после ранжирования модели: ', accuracy_top_n(valid_target['Expected'], best_predicted_neighbors))

accuracy@5 после ранжирования модели:  73.871


In [53]:
df_best_predicted_neighbors = pd.DataFrame(best_predicted_neighbors, index=valid_features.index)

In [54]:
df_best_predicted_neighbors = (df_best_predicted_neighbors
                               .apply(lambda x: x.apply(lambda x: str(int(x)) + '-base')))
df_best_predicted_neighbors

,0,1,2,3,4
Id,,,,,
100000-query,1908173-base,2512787-base,2546410-base,2875559-base,270881-base
100001-query,2259158-base,1079425-base,801389-base,2197829-base,1068829-base
100002-query,431806-base,148400-base,451870-base,365989-base,1783635-base
100003-query,2171299-base,2140047-base,1579116-base,323952-base,1512255-base
100004-query,74247-base,1003744-base,1733365-base,608523-base,1446303-base
...,...,...,...,...,...
199995-query,1876024-base,184973-base,1403690-base,2726244-base,827859-base
199996-query,1895629-base,2063426-base,120969-base,841235-base,988237-base
199997-query,918008-base,581956-base,2249505-base,2341431-base,232504-base


## Итог поиска ближайших векторов методом кластеризации с применением ранжирующей модели машинного обучения

В качестве ранжирующей модели машинного обучения выбрана модель CatBoostClassifier. 

Исследована возможность повышения точности поиска путем применения ранжирующей модели машинного обучения для топ 50, 100, 200, 400 ближайших векторов. ПРименение модели на данных выборка дает сравнимый результат точности, но увеличение числа ближайших векторов приводит к увеличению времени обучения и предсказания модели. 

Эмпирическим путем в качестве оптимального варианта для ранжирования была выбрана выборка из топ 100 векторов, отобранных методом кластеризации из библиотеки Faiss.    

Для модели машинного обучения удаление признаков с расределением, отличным от нормального, приводит к существенному (около 10%) снижению качества ранжирования.  

Проведен подбор гиперпараметров модели с использованием optuna. Проведено итерационное дообучение модели на всем объеме тренировочной выборки.  

Дополнительное применение модели машинного обучения на выбоке из топ 100 ближайших векторов ориентировочно увеличивает точность поиска на 3%.  